In [1]:
from xml.etree import ElementTree as ET
import pandas as pd
from datetime import datetime

# Caminho para o arquivo GPX fornecido
gpx_file_path = '20240409-101422 - Trajeto de teste 1 09_04.gpx'

# Caminho para o arquivo data.csv
data_csv_path = 'data.csv'

# Ler o arquivo GPX
with open(gpx_file_path, 'r') as gpx_file:
    gpx_tree = ET.parse(gpx_file)
    root = gpx_tree.getroot()

# Namespace para o arquivo GPX
namespace = 'http://www.topografix.com/GPX/1/0'

# Encontrar trilhas no arquivo GPX
tracks = root.findall(f'{{{namespace}}}trk')

# Encontrar segmentos na primeira trilha
trkseg = tracks[0].find(f'{{{namespace}}}trkseg')

# Extrair pontos de trilha
trkpts = trkseg.findall(f'{{{namespace}}}trkpt')

# Extrair informações de cada ponto de trilha, incluindo velocidade (speed) e satélites (sat)
trkpt_data = []
for pt in trkpts:
    lat = pt.attrib['lat']
    lon = pt.attrib['lon']
    ele = pt.find(f'{{{namespace}}}ele').text if pt.find(f'{{{namespace}}}ele') is not None else None
    speed = pt.find(f'{{{namespace}}}speed').text if pt.find(f'{{{namespace}}}speed') is not None else None
    sat = pt.find(f'{{{namespace}}}sat').text if pt.find(f'{{{namespace}}}sat') is not None else None
    time_text = pt.find(f'{{{namespace}}}time').text if pt.find(f'{{{namespace}}}time') is not None else None
    time = datetime.strptime(time_text, '%Y-%m-%dT%H:%M:%SZ') if time_text else None
    trkpt_data.append({
        'latitude': lat,
        'longitude': lon,
        'elevation': ele,
        'speed': speed,
        'satellites': sat,
        'time': time
    })

# Converter para DataFrame
trkpt_df = pd.DataFrame(trkpt_data)

# Diminuir 3 horas da coluna 'time'
trkpt_df['time'] = trkpt_df['time'] - pd.Timedelta(hours=3)

# Separar a coluna 'time' em 'date' e 'hour'
trkpt_df['date'] = trkpt_df['time'].dt.date
trkpt_df['hour'] = trkpt_df['time'].dt.time

# Exibir as primeiras linhas para verificar
print(trkpt_df.head())


      latitude     longitude elevation  speed satellites                time  \
0  -3.74468661  -38.57741007    16.312  0.000         32 2024-04-09 10:14:22   
1  -3.74468491  -38.57741068    16.476  0.000         32 2024-04-09 10:14:23   
2  -3.74468784  -38.57741192    16.121  0.000         32 2024-04-09 10:14:24   
3  -3.74468931  -38.57741166    15.629  0.000         32 2024-04-09 10:14:25   
4  -3.74469007  -38.57741203    15.846  0.000         33 2024-04-09 10:14:26   

         date      hour  
0  2024-04-09  10:14:22  
1  2024-04-09  10:14:23  
2  2024-04-09  10:14:24  
3  2024-04-09  10:14:25  
4  2024-04-09  10:14:26  


In [2]:

# Ler o arquivo data.csv
data_csv_df = pd.read_csv(data_csv_path, sep=',')

# Corrigir o nome das colunas, removendo espaços extras
data_csv_df.columns = data_csv_df.columns.str.strip()

# Converter RTCData e RTCHora para o mesmo formato de trkpt_df, tratando datas e horas inválidas
data_csv_df['RTCData'] = pd.to_datetime(data_csv_df['RTCData'], errors='coerce', dayfirst=True).dt.date
data_csv_df['RTCHora'] = pd.to_datetime(data_csv_df['RTCHora'], errors='coerce', format='%H:%M:%S').dt.time

# Realizar o merge com base em 'RTCData', 'RTCHora' e as colunas 'date', 'hour' de trkpt_df
merged_df = pd.merge(data_csv_df, trkpt_df[['latitude', 'longitude', 'elevation', 'date', 'hour', 'speed', 'satellites']],
                     left_on=['RTCData', 'RTCHora'], right_on=['date', 'hour'], how='left')


# Renomear colunas para corresponder aos nomes especificados
merged_df.rename(columns={'latitude': 'LatLogger', 'longitude': 'LongLogger',
                          'elevation': 'ElevationLogger', 'date': 'DataLogger', 'hour': 'HoraLogger', 'speed': 'SpeedLogger', 'satellites': 'SatellitesLogger'},
                 inplace=True)

# Definir o caminho do arquivo de destino
caminho_arquivo_csv = 'datacsv2.csv' # Ajuste o caminho conforme necessário
# Exportar o DataFrame para um arquivo CSV
data_csv_df.to_csv(caminho_arquivo_csv, sep=';', encoding='ISO-8859-1', index=False)

C:\Users\jairo\AppData\Local\Temp\ipykernel_60692\3562980708.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_csv_df['RTCData'] = pd.to_datetime(data_csv_df['RTCData'], errors='coerce', dayfirst=True).dt.date


In [3]:
merged_df.head(), merged_df.shape

(      RTCData   RTCHora GPSData GPSHora  Lat  Long  Altgps  Vel  GPSUpdate  \
 0         NaT  00:00:00   0/0/0   0:0:0  NaN   NaN     0.0  0.0          0   
 1  2024-04-09  09:52:12   0/0/0   0:0:0  NaN   NaN     0.0  0.0          0   
 2  2024-04-09  09:52:12   0/0/0   0:0:0  NaN   NaN     0.0  0.0          0   
 3  2024-04-09  09:52:12   0/0/0   0:0:0  NaN   NaN     0.0  0.0          0   
 4  2024-04-09  09:52:12   0/0/0   0:0:0  NaN   NaN     0.0  0.0          0   
 
      Ax  ...    Gy    Gz  indiceAmostra  LatLogger  LongLogger  \
 0  0.00  ...  0.00  0.00              0        NaN         NaN   
 1  0.01  ...  0.65 -1.65              1        NaN         NaN   
 2  0.01  ...  0.95 -1.53              2        NaN         NaN   
 3  0.01  ...  0.77 -1.34              3        NaN         NaN   
 4  0.01  ...  0.89 -1.46              4        NaN         NaN   
 
    ElevationLogger DataLogger HoraLogger SpeedLogger SatellitesLogger  
 0              NaN        NaN        NaN      

In [4]:
# Definir o caminho do arquivo de destino
caminho_arquivo_csv = 'gpsMergeData.csv' # Ajuste o caminho conforme necessário
# Exportar o DataFrame para um arquivo CSV
merged_df.to_csv(caminho_arquivo_csv, sep=';', encoding='ISO-8859-1', index=False)

PermissionError: [Errno 13] Permission denied: 'gpsMergeData.csv'

In [ ]:
# Drop all rows where the column 'DataLogger' has missing values
merged_df_cleaned = merged_df.dropna(subset=['DataLogger'])

# Display the first few rows of the cleaned DataFrame to confirm the changes
merged_df_cleaned.head()


In [ ]:
import pandas as pd
import numpy as np

# Supondo que o DataFrame 'merged_df' já foi carregado

# Criar uma cópia limpa do DataFrame para trabalhar
merged_df_cleaned = merged_df.dropna(subset=['DataLogger']).copy()

# Converter 'RTCHora' para string
merged_df_cleaned['RTCHora'] = merged_df_cleaned['RTCHora'].astype(str)

# Adicionar a coluna 'Seconds'
merged_df_cleaned['Seconds'] = pd.to_timedelta(merged_df_cleaned['RTCHora']).dt.total_seconds()

# Assegurar que 'LatLogger' e 'LongLogger' são numéricos e tratar NaNs
merged_df_cleaned['LatLogger'] = pd.to_numeric(merged_df_cleaned['LatLogger'], errors='coerce')
merged_df_cleaned['LongLogger'] = pd.to_numeric(merged_df_cleaned['LongLogger'], errors='coerce')

# Preencher valores NaN para evitar erros na interpolação
merged_df_cleaned['LatLogger'].fillna(method='ffill', inplace=True)
merged_df_cleaned['LongLogger'].fillna(method='ffill', inplace=True)

# Agrupar por segundos
grouped = merged_df_cleaned.groupby('Seconds')

lat_inter_list = []
long_inter_list = []

for _, group in grouped:
    first_lat = group['LatLogger'].iloc[0]
    first_long = group['LongLogger'].iloc[0]
    
    try:
        next_lat = grouped.get_group(_ + 1)['LatLogger'].iloc[0]
        next_long = grouped.get_group(_ + 1)['LongLogger'].iloc[0]
    except KeyError:
        next_lat = first_lat
        next_long = first_long
    
    num_points = len(group)
    
    if num_points > 1:  # Só interpola se há mais de um ponto no segundo
        lat_seq = np.linspace(first_lat, next_lat, num_points + 1)[:-1]
        long_seq = np.linspace(first_long, next_long, num_points + 1)[:-1]
    else:  # Se só houver um ponto, usa-se o valor existente
        lat_seq = [first_lat]
        long_seq = [first_long]
    
    lat_inter_list.extend(lat_seq)
    long_inter_list.extend(long_seq)

# Atualizar o DataFrame
merged_df_cleaned.loc[:, 'LatInter'] = lat_inter_list
merged_df_cleaned.loc[:, 'LongInter'] = long_inter_list

# Verificar os resultados
print(merged_df_cleaned[['RTCHora', 'LatLogger', 'LongLogger', 'SpeedLogger', 'LatInter', 'LongInter']].head())


In [ ]:
# Definir o caminho do arquivo de destino
caminho_arquivo_csv = 'gpsMergeDataCleanerInter.csv' # Ajuste o caminho conforme necessário
# Exportar o DataFrame para um arquivo CSV
merged_df_cleaned.to_csv(caminho_arquivo_csv, sep=';', encoding='ISO-8859-1', index=False)